In [1]:
#!python -m tf2onnx.convert --saved-model trained_models/mlpTest/ --output mlp.onnx

Run with trackml

In [1]:
import tensorflow as tf
network = "mlpTest"

2024-06-18 22:26:15.285586: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 22:26:15.285610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 22:26:15.286641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 22:26:15.291713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 22:26:15.813457: W tensorflow/compiler/tf2

In [2]:
init_model = tf.keras.models.load_model(f"../models/tensorflow/{network}/")

2024-06-18 22:26:16.207747: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 22:26:16.258619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 22:26:16.258800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
window_size = 3
total_lenght = 30

In [34]:
nn_input = tf.keras.layers.Input(shape=(window_size, 3), name="input_1")
# Run the model on the rolling window of 3 rows
_x = init_model(nn_input)
nn_output = tf.keras.layers.concatenate((nn_input, tf.expand_dims(_x, axis=1)), axis=1)
for i in range(1,total_lenght-window_size):
    _x = init_model(tf.slice(nn_output, begin=[0, i, 0], size=[-1, window_size, 3]))
    nn_output = tf.keras.layers.concatenate((nn_output, tf.expand_dims(_x, axis=1)), axis=1)
model = tf.keras.models.Model(inputs=nn_input, outputs=nn_output)


In [35]:
x = tf.constant([[[1,2,3],[4,5,6],[7,8,9]]], dtype=tf.float32)
y = tf.constant([[1,2,3]], dtype=tf.float32)

In [46]:
#%timeit -n 100000 tf.expand_dims(y, axis=0) # 22.1 µs ± 49.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
#%timeit -n 100000 tf.reshape(y, (1,3)) # 74.7 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

22.1 µs ± 49.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
74.7 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [51]:
#%timeit -n 100000 tf.slice(x, begin=[0, 1, 0], size=[-1, 2, 3]) # 122 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
#%timeit -n 100000 x[:,1:3,:] # 179 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

122 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
179 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3, 3)]               0         []                            
                                                                                                  
 sequential (Sequential)     (None, 3)                    4611      ['input_1[0][0]',             
                                                                     'tf.slice_78[0][0]',         
                                                                     'tf.slice_79[0][0]',         
                                                                     'tf.slice_80[0][0]',         
                                                                     'tf.slice_81[0][0]',         
                                                                     'tf.slice_82[0][0]',   

In [22]:
import tf2onnx

In [23]:
tf2onnx.convert.from_keras(model, output_path=f"../models/onnx/{network}_track.onnx")

2024-06-18 22:41:10.285292: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 22:41:10.285427: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-06-18 22:41:10.285490: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-06-18 22:41:10.285679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 22:41:10.285813: I external/local_xla/xla/stream_executor/cuda/cuda_e

(ir_version: 8
 producer_name: "tf2onnx"
 producer_version: "1.16.1 15c810"
 graph {
   node {
     input: "input_1"
     input: "const_fold_opt__814"
     output: "model_3/sequential/flatten/Reshape:0"
     name: "model_3/sequential/flatten/Reshape"
     op_type: "Reshape"
   }
   node {
     input: "model_3/sequential/flatten/Reshape:0"
     input: "model_3/sequential/dense/MatMul_8/ReadVariableOp:0"
     output: "model_3/sequential/dense/MatMul:0"
     name: "model_3/sequential/dense/MatMul"
     op_type: "MatMul"
   }
   node {
     input: "model_3/sequential/dense/MatMul:0"
     input: "model_3/sequential/dense/BiasAdd_18/ReadVariableOp:0"
     output: "model_3/sequential/dense/BiasAdd:0"
     name: "model_3/sequential/dense/BiasAdd"
     op_type: "Add"
   }
   node {
     input: "model_3/sequential/dense/BiasAdd:0"
     output: "model_3/sequential/dense/Relu:0"
     name: "model_3/sequential/dense/Relu"
     op_type: "Relu"
   }
   node {
     input: "model_3/sequential/dense/Rel